# PostgreSQL テーブル確認ノートブック
このノートは `db_config.py` を読み込み、PostgreSQL からテーブル内容を確認します。
- 接続情報: `db_config.py`（作業ディレクトリ直下 / `nf_featgen_bundle_v1/` / `db/` いずれか）
- 代表テーブル: `public.features_hist`, `public.features_futr`

任意で `TABLE_SCHEMA`, `TABLE_HIST`, `TABLE_FUTR` を変更して使ってください。

In [1]:
# 0) セットアップ
import os, sys, importlib
from pathlib import Path
import pandas as pd
from sqlalchemy import create_engine, text
from urllib.parse import quote_plus
import matplotlib.pyplot as plt
print(pd.__version__)


2.3.3


In [2]:
# 1) db_config.py のロード
def _load_db_config():
    for p in [Path.cwd()/"db_config.py", Path.cwd()/"nf_featgen_bundle_v1"/"db_config.py", Path.cwd()/"db"/"db_config.py"]:
        if p.exists():
            spec = importlib.util.spec_from_file_location("db_config", str(p))
            mod = importlib.util.module_from_spec(spec)
            spec.loader.exec_module(mod)
            print("Loaded:", p)
            return mod
    raise FileNotFoundError("db_config.py が見つかりません。")

db_cfg = _load_db_config()
DB_CONFIG = db_cfg.DB_CONFIG
TABLE_PREFIX = getattr(db_cfg, "TABLE_PREFIX", "")
DB_CONFIG


Loaded: /mnt/e/env/ts/test/db_config.py


{'user': 'postgres',
 'password': 'z',
 'host': 'localhost',
 'port': 5432,
 'dbname': 'postgres'}

In [3]:
# 2) エンジン作成
def make_url(cfg: dict) -> str:
    if "url" in cfg and cfg["url"]:
        return cfg["url"]
    user = cfg.get("user") or cfg.get("username")
    pwd  = cfg.get("password") or cfg.get("pwd") or cfg.get("pass")
    host = cfg.get("host", "localhost")
    port = int(cfg.get("port", 5432))
    db   = cfg.get("dbname") or cfg.get("database") or cfg.get("db")
    if not (user and pwd and db):
        raise ValueError(f"DB設定を確認してください。渡されたキー: {list(cfg.keys())}")
    return f"postgresql+psycopg2://{quote_plus(user)}:{quote_plus(pwd)}@{host}:{port}/{db}"

engine = create_engine(make_url(DB_CONFIG), pool_pre_ping=True, future=True)
with engine.begin() as conn:
    print(conn.execute(text("SELECT current_user, current_database();")).all())


[('postgres', 'postgres')]


In [5]:
# 3) ユーティリティ
import pandas as pd
from sqlalchemy import text

def q(sql, params=None, limit=None):
    if limit is not None and "limit" not in sql.lower():
        sql = sql.rstrip(";") + f" LIMIT {int(limit)}"
    return pd.read_sql(text(sql), engine, params=params)

def list_tables_like(pattern="features%"):
    sql = '''
    SELECT table_schema, table_name
    FROM information_schema.tables
    WHERE table_type='BASE TABLE' AND table_schema NOT IN ('pg_catalog','information_schema')
      AND table_name ILIKE :pat
    ORDER BY table_schema, table_name
    '''
    return q(sql, {"pat": pattern})

def table_preview(table, limit=5):
    return q(f"SELECT * FROM {table} ORDER BY 1", limit=limit)

def table_cols(table):
    sql = '''
    SELECT c.ordinal_position AS pos, c.column_name, c.data_type, c.is_nullable
    FROM information_schema.columns c
    WHERE (c.table_schema || '.' || c.table_name) = :tbl
    ORDER BY c.ordinal_position
    '''
    return q(sql, {"tbl": table})

def count_rows(table):
    return q(f"SELECT COUNT(*) AS n FROM {table}")
    
TABLE_SCHEMA = "public"
TABLE_HIST   = f"{TABLE_SCHEMA}.features_hist"
TABLE_FUTR   = f"{TABLE_SCHEMA}.features_futr"

list_tables_like("features%")


,table_schema,table_name
0,public,features_futr
1,public,features_hist


In [6]:
# 4) 先頭5行のプレビュー
table_preview(TABLE_HIST, limit=5)


,unique_id,ds,hist_y_lag1,hist_y_lag7,hist_y_lag14,hist_y_lag28,hist_y_roll_mean_7,hist_y_roll_std_7,hist_y_roll_mean_28,hist_y_diff_1,...,futr_fourier_P365_s1,futr_fourier_P365_c1,futr_fourier_P365_s2,futr_fourier_P365_c2,futr_fourier_P365_s3,futr_fourier_P365_c3,futr_fourier_P365_s4,futr_fourier_P365_c4,futr_fourier_P365_s5,futr_fourier_P365_c5
0,N1,1999-04-27,1.0,None,None,None,NaN,NaN,None,1.0,...,-0.796183,-0.605056,0.963471,-0.267814,-0.369725,0.929141,-0.516062,-0.856551,0.994218,0.107381
1,N1,1999-05-11,2.0,None,None,None,1.500000,0.707107,None,-1.0,...,-0.917584,-0.397543,0.729558,-0.683919,0.337523,0.941317,-0.997917,-0.064508,0.455907,-0.890028
2,N1,1999-05-25,1.0,None,None,None,1.333333,0.577350,None,10.0,...,-0.985948,-0.167052,0.329408,-0.944188,0.875892,0.482508,-0.622047,0.782980,-0.668064,-0.744104
3,N1,1999-06-08,11.0,None,None,None,3.750000,4.856267,None,-3.0,...,-0.997325,0.073095,-0.145799,-0.989314,0.976011,-0.217723,0.288482,0.957485,-0.933837,0.357698
4,N1,1999-04-13,NaN,None,None,None,NaN,NaN,None,NaN,...,-0.628763,-0.777597,0.977848,0.209315,-0.891981,0.452072,0.409356,-0.912375,0.255353,0.966848


In [7]:
table_preview(TABLE_FUTR, limit=5)


,unique_id,ds,futr_year,futr_quarter,futr_month,futr_weekofyear,futr_day,futr_dayofweek,futr_is_month_start,futr_is_month_end,...,futr_fourier_P365_s1,futr_fourier_P365_c1,futr_fourier_P365_s2,futr_fourier_P365_c2,futr_fourier_P365_s3,futr_fourier_P365_c3,futr_fourier_P365_s4,futr_fourier_P365_c4,futr_fourier_P365_s5,futr_fourier_P365_c5
0,N1,2025-11-14,2025,4,11,46,14,4,0,0,...,0.977848,0.209315,0.409356,-0.912375,-0.806480,-0.591261,-0.746972,0.664855,0.493776,0.869589
1,N1,2025-11-15,2025,4,11,46,15,5,0,0,...,0.981306,0.192452,0.377708,-0.925925,-0.835925,-0.548843,-0.699458,0.714673,0.566702,0.823923
2,N1,2025-11-16,2025,4,11,46,16,6,0,0,...,0.984474,0.175531,0.345612,-0.938377,-0.863142,-0.504961,-0.648630,0.761104,0.635432,0.772157
3,N1,2025-11-17,2025,4,11,47,17,0,0,0,...,0.987349,0.158559,0.313107,-0.949718,-0.888057,-0.459733,-0.594727,0.803928,0.699458,0.714673
4,N1,2025-11-13,2025,4,11,46,13,3,0,0,...,0.974100,0.226116,0.440519,-0.897743,-0.774884,-0.632103,-0.790946,0.611886,0.417194,0.908818


In [8]:
# 5) 行数・列情報
count_rows(TABLE_HIST), count_rows(TABLE_FUTR)


(       n
 0  27217,
      n
 0  224)

In [9]:
table_cols(TABLE_HIST)


,pos,column_name,data_type,is_nullable
0,1,unique_id,text,YES
1,2,ds,timestamp without time zone,YES
2,3,hist_y_lag1,double precision,YES
3,4,hist_y_lag7,double precision,YES
4,5,hist_y_lag14,double precision,YES
5,6,hist_y_lag28,double precision,YES
6,7,hist_y_roll_mean_7,double precision,YES
7,8,hist_y_roll_std_7,double precision,YES
8,9,hist_y_roll_mean_28,double precision,YES
9,10,hist_y_diff_1,double precision,YES


In [10]:
table_cols(TABLE_FUTR)


,pos,column_name,data_type,is_nullable
0,1,unique_id,text,YES
1,2,ds,timestamp without time zone,YES
2,3,futr_year,integer,YES
3,4,futr_quarter,integer,YES
4,5,futr_month,integer,YES
5,6,futr_weekofyear,bigint,YES
6,7,futr_day,integer,YES
7,8,futr_dayofweek,integer,YES
8,9,futr_is_month_start,smallint,YES
9,10,futr_is_month_end,smallint,YES


In [11]:
# 6) 時系列プロット（matplotlibのみ / 単独プロット / 色指定なし）
uid = q(f"SELECT DISTINCT unique_id FROM {TABLE_HIST} ORDER BY 1 LIMIT 1").iloc[0,0]
df = q(f"""SELECT unique_id, ds, y
FROM {TABLE_HIST}
WHERE unique_id = :uid
ORDER BY ds
""" , {"uid": uid})
print("unique_id:", uid, "rows:", len(df))

import matplotlib.pyplot as plt
plt.figure()
plt.plot(df["ds"], df["y"])  # 色は指定しない
plt.title(f"y over time - {uid}")
plt.xlabel("ds")
plt.ylabel("y")
plt.show()


ProgrammingError: (psycopg2.errors.UndefinedColumn) column "y" does not exist
LINE 1: SELECT unique_id, ds, y
                              ^

[SQL: SELECT unique_id, ds, y
FROM public.features_hist
WHERE unique_id = %(uid)s
ORDER BY ds
]
[parameters: {'uid': 'N1'}]
(Background on this error at: https://sqlalche.me/e/20/f405)